In [11]:
import pandas as pd
import sys
import os

# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
statusDir = projectDir + "source-data/status-codes/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

sdataDir = "current-lists/sensitive-lists/"
vicchecklisturl = "https://vba.dse.vic.gov.au/vba/downloadVSC.do" # gets a csv
vicchecklist = pd.read_csv(projectDir + "source-data/VIC/Species-Checklist.csv",encoding='cp1252')

# Extract and format initial list
1. Drop non-required columns
2. Rename columns for Darwin Core list terms
3. Assign additional required list fields

In [12]:
vicchecklist['LAST_MOD'].drop_duplicates().sort_values()

22182    01/01/2010
10432    01/02/2012
11265    01/02/2018
12493    01/02/2021
18298    01/03/2017
            ...    
13112    31/10/2016
159      31/10/2017
10501    31/10/2018
11206    31/10/2020
15567    31/10/2022
Name: LAST_MOD, Length: 899, dtype: object

In [13]:
# Drop columns
vicchecklist = vicchecklist.drop(['PARENT_TAXON_LEVEL_CDE','PARENT_TAXON_ID','ALL_DISCIPLINE_CODES',
                                          'TREATIES','SHORT_NAME','NVIS_GROWTHFORM','SCIENTIFIC_NME_SYNONYM',
                                          'COMMON_NME_SYNONYM','PRINT_ORDER_NUM', 'VIC_ADVISORY_STATUS', 'ORIGIN'],axis=1)
# Rename columns
vicchecklist = vicchecklist.rename(columns=
{
    'SCIENTIFIC_NAME':'scientificName',
    'FFG_ACT_STATUS':'ffgactstatus',
    'EPBC_ACT_STATUS': 'epbcactStatus',
    'TAXON_LEVEL_CDE': 'taxonRank',
    'ORIGIN': 'establishmentMeans',
    'COMMON_NME_SYNONYM': 'taxonRemarks',
    'COMMON_NAME':'vernacularName',
    'AUTHORITY':'scientificNameAuthority',
    'PRIMARY_DISCIPLINE': 'primaryDiscipline',
    'RESTRICTED_FLAG' : 'restrictedFlag',
    'EXTRACT_DATE': 'extractDate',
    'LAST_MOD':'modified'
})
vicchecklist['status'] = vicchecklist['ffgactstatus'].str.replace('(Extinct in Victoria)', '',  regex=False)
vicchecklist['taxonRank'] = 'species'
vicchecklist['status'] = vicchecklist['ffgactstatus']
vicchecklist['sourceStatus'] = vicchecklist['ffgactstatus']

# reformat date
vicchecklist['extractDate'] = pd.to_datetime(vicchecklist['extractDate'], format='%d/%m/%Y')
vicchecklist['extractDate'] = vicchecklist['extractDate'].dt.strftime('%Y-%m-%d')
vicchecklist['modified'] = pd.to_datetime(vicchecklist['modified'], format='%d/%m/%Y')
vicchecklist['modified'] = vicchecklist['modified'].dt.strftime('%Y-%m-%d')

In [14]:
vicchecklist['status'].unique()

array([nan, 'Extinct', 'Endangered', 'Critically Endangered',
       'Vulnerable', 'Threatened', 'Conservation Dependent',
       'Endangered (Extinct in Victoria)'], dtype=object)

In [15]:
vicchecklist['status'] = vicchecklist['status'].str.replace('(Extinct in Victoria)', '',  regex=False)
vicchecklist['status'] = vicchecklist['status'].str.strip()

# Split out sensitive and conservation lists

In [16]:
sensitivelist = vicchecklist[vicchecklist["restrictedFlag"].notna()]

d = {'sensitive list count': [len(sensitivelist.index)]}
pd.DataFrame(d).transpose()

,0
sensitive list count,136
conservation list count,1999


## Process Sensitive List

In [17]:
sensitivelist['status'].unique()

array(['Critically Endangered', 'Vulnerable', 'Endangered', nan],
      dtype=object)

In [18]:
sensitivelist = sensitivelist[['scientificName', 'vernacularName']]
sensitivelist['generalisation'] = "1 km"

In [19]:
sensitivelist.to_csv(projectDir + sdataDir + "VIC-sensitive-20230601.csv",  encoding="UTF-8", index=False)
print('Finished writing VIC Sensitive list')

Finished processing
